# Numerical Simulation

In [1]:
import numpy as np

In [2]:
def is_hermitian(matrix):
    return np.allclose(matrix, matrix.conj().T)

In [115]:
off_d = 1/3
on_d = 1/2

Let's write some code to make a Hermitian matrix and get its eigenvalues:

In [120]:
off_d = 1/5
on_d = 1

b = np.array([1, 0])

A = np.array(
    [[on_d, off_d], 
     [off_d, on_d]]
)

print(f"A = {A}")
print(f"b = {b}")

print(f"Is A Hermitian? {is_hermitian(A)}")

eigenvalues, eigenvectors = np.linalg.eig(A)

eig_ratio = eigenvalues[0] / eigenvalues[1]
eig_ratio = np.round(eig_ratio, 3)

print(f"The eigenvalues are {eigenvalues}")
print(f"The ratio of the eigenvalues is {eig_ratio}")

A = [[1.  0.2]
 [0.2 1. ]]
b = [1 0]
Is A Hermitian? True
The eigenvalues are [1.2 0.8]
The ratio of the eigenvalues is 1.5


The ratio of our eigenvalues means that we need to encode them in $2$ quantum states whose ratio is the same. We can do this as follows:

$$
e_0 = |10\rangle \quad (=2) \\
\, \\
e_1 = |11\rangle \quad (=3)
$$

Which means we'll only need to use $2$ qubits for our clock register.


For $t$, we have to choose something so that our $\tilde{\lambda_j}$'s become integers.

Given the formula $\tilde{\lambda_j} = \frac{2^n \lambda_j t}{2\pi}$, knowing that our $2^n = 4$, we need to choose $t$ such that it cancels out this $4$ term, the $\pi$ in the denominator and the least common denominator of our $\lambda_j$'s, which is $\frac{1}{5}$, as our eigenvalues are:

$$
\lambda_0 = \frac{6}{5} \\ 
\, \\
\lambda_1 = \frac{4}{5}
$$

Therefore, we pick $t=\frac{5 \pi}{4}$, which gives us:

$$
\tilde{\lambda_0} = 2 \Rightarrow |\tilde{\lambda_0} \rangle =  |10\rangle \\
\, \\
\tilde{\lambda_1} = 3 \Rightarrow |\tilde{\lambda_1} \rangle =  |11\rangle
$$

Very nice!

Let's also just assume that 
$$
|b\rangle = \begin{pmatrix} 0 \\ 1 \end{pmatrix}
$$

Let's find the $U$ matrix that we want to use in our Quantum Phase Estimation algorithm. Since we have $2$ clock qubits, we only need to apply $U^2$ and $U$.

In [123]:
t = 5 * np.pi / 4 

U = np.exp(-1j * t * A)

U2 = U @ U

print(f"U = {U}")
print()
print(f"U2 = {U2}")


U = [[-0.70710678+0.70710678j  0.70710678-0.70710678j]
 [ 0.70710678-0.70710678j -0.70710678+0.70710678j]]

U2 = [[ 3.45913843e-16-2.j -3.13396163e-16+2.j]
 [-3.13396163e-16+2.j  2.80878484e-16-2.j]]


And we want to create the controlled versions of these unitaries, so we make them using this formula:

$$
CU = \begin{pmatrix} I & 0 \\ 0 & U \end{pmatrix}
$$

In [127]:
def make_controlled_U(U):
    n = U.shape[0]
    return np.block([[np.eye(n), np.zeros((n, n))], 
                     [np.zeros((n, n)), U]])

In [128]:
c_U = make_controlled_U(U)
c_U2 = make_controlled_U(U2)

print(f"c_U = {c_U}")
print()
print(f"c_U2 = {c_U2}")


c_U = [[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          1.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.70710678+0.70710678j
   0.70710678-0.70710678j]
 [ 0.        +0.j          0.        +0.j          0.70710678-0.70710678j
  -0.70710678+0.70710678j]]

c_U2 = [[ 1.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  1.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  3.45913843e-16-2.j
  -3.13396163e-16+2.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j -3.13396163e-16+2.j
   2.80878484e-16-2.j]]


In [132]:
eigenvectors

array([[ 0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678]])

So we know our eigenvectors are:

$$
\vec{e_0} = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 \\ -1 \end{pmatrix} \\
\, \\
\Rightarrow |e_0 \rangle = | - \rangle \\
\, \\
\vec{e_1} = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 \\ 1 \end{pmatrix} \\
\, \\
\Rightarrow |e_1 \rangle = | + \rangle
$$

Which means our $|b\rangle$ state can be written as:

$$
|b\rangle = \begin{pmatrix} 0 \\ 1 \end{pmatrix} = \frac{1}{\sqrt{2}} \times \vec{e_0} + \frac{1}{\sqrt{2}} \times \vec{e_1}
$$

Which means:

$$
b_0 = \frac{1}{\sqrt{2}}, \quad b_1 = \frac{1}{\sqrt{2}}
$$

Step 1.

After applying the QPE with the controlled versions of our $U$ and $U^2$ matrices and with our chosen $t$, our state becomes:

$$
\frac{1}{\sqrt{2}}  \times |- \rangle |10\rangle |0\rangle + \frac{1}{\sqrt{2}}  \times |+ \rangle |11\rangle |0\rangle
$$

(having written down how QPE works and its internal workings, we'll omit the numerical calculations for brevity and just use the final state)

Knowing our $\tilde{\lambda_j}$'s, we choose $C=1$ to find the angle required for our $R_y$ gate.

Remembering how our state should look like after the rotation:

$$
\sum_{j=0}^{N-1} b_j | u_j \rangle | \tilde{\lambda_j} \rangle  \Big( \sqrt{1 - \frac{C^2}{\tilde{\lambda_j}^2}} | 0 \rangle _a + \frac{C}{\tilde{\lambda_j}} | 1 \rangle_a \Big)
$$

This means also $C \leq \tilde{\lambda_j}$. Since the minimal $\tilde{\lambda_j}$ is $1$, we will set $C = 1$ to maximize the probability of measuring $| 1 \rangle$ during the ancilla bit measurement. 

The formula for the angle is:

$$
\theta_j = 2\cos^{-1}(\frac{C}{\tilde{\lambda_j}})
$$

Which means:

In [133]:
theta_0 = 2 * np.arccos(1/2)
theta_1 = 2 * np.arccos(1/3)

print(f"theta_0 = {theta_0}")
print(f"theta_1 = {theta_1}")

theta_0 = 2.0943951023931957
theta_1 = 2.4619188346815495


Now, for executing the rotation, we need to control it with our eigenstates, so our total rotation gate becomes:

$$
Rot = |10\rangle \langle 10| \otimes R_y(\theta_0) + |11\rangle \langle 11| \otimes R_y(\theta_1)
$$

Where the control is our clock register and the target is our ancilla qubit.

Step 2.

So after the rotation, our state becomes:

$$
\begin{align*} 
&  \frac{1}{\sqrt{2}}  \times |- \rangle |10\rangle \Big( \sqrt{\frac{3}{4}} |0 \rangle + \frac{1}{2} | 1 \rangle \Big) \, +  \\
& \frac{1}{\sqrt{2}}  \times |+ \rangle |11\rangle \Big( \sqrt{\frac{8}{9}} |0 \rangle + \frac{1}{3} | 1 \rangle \Big)
\end{align*}
$$

Step 3.

Now we perform the inverse QPE on our clock and $b$ register to put the clock register back to $|0^{n}\rangle$ state and the total state becomes:

$$
\begin{align*} 
&  \frac{1}{\sqrt{2}}  \times |- \rangle |00\rangle \Big( \sqrt{\frac{3}{4}} |0 \rangle + \frac{1}{2} | 1 \rangle \Big) \, +  \\
& \frac{1}{\sqrt{2}}  \times |+ \rangle |00\rangle \Big( \sqrt{\frac{8}{9}} |0 \rangle + \frac{1}{3} | 1 \rangle \Big)
\end{align*}
$$

Step 4. 

Now we measure and assuming that we've measured $|1\rangle$ on our ancilla qubit, our state becomes:
(omitting the normalization factor and the clock qubits)

$$
\frac{1}{\sqrt{2}} \times \frac{1}{2} |- \rangle |1\rangle
+ \frac{1}{\sqrt{2}} \times \frac{1}{3} |+ \rangle |1\rangle
$$

In [151]:
x = np.linalg.inv(A) @ b

minus_state = np.array([1, -1]) / np.sqrt(2)
plus_state = np.array([1, 1]) / np.sqrt(2)

x_hhl = 0.5 * minus_state + 1/3 * plus_state

We see that the real $x$ that we would obtain by inverting the matrix and multiplying it with our $b$ vector is:

In [150]:
print(f"x = {x}")

x = [ 1.04166667 -0.20833333]


And the answer we get from our HHL algorithm is:

In [152]:
print(f"hhl = {x_hhl}")

hhl = [ 0.58925565 -0.11785113]


To see that they're equivalent by a constant factor, we can element-wise divide the two vectors and see that:

In [153]:
x / x_hhl

array([1.76776695, 1.76776695])

Which means that the constant factor is:
$$
1.76776695
$$

And we indeed get the correct answer by running the HHL algorithm!